<a href="https://colab.research.google.com/github/YeChen/RevealAPITest/blob/main/reveal_api_test_3_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import os
from google.colab import userdata
from IPython.display import display, HTML

# Step 1: Retrieve USER and PASSWORD secrets from Colab
username = userdata.get("DEMO_USER")
password = userdata.get("DEMO_PASSWORD")

if not username or not password:
    raise ValueError("Secrets USER and PASSWORD must be set in Colab.")

def print_color(message, color):
    display(HTML(f'<p style="color: {color};">{message}</p>'))

# Step 2: Define the authentication function
def authenticate(username, password):
    """
    Authenticates with the Reveal API using username and password.
    Returns loginSessionId and userId if authentication is successful.
    """
    login_url = "https://consulting.us-east-1.reveal11.cloud/rest/api/v2/login"
    login_data = {
        "username": username,
        "password": password
    }

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Reveal-API-Tester/1.0"
    }

    response = requests.post(login_url, json=login_data, headers=headers)
    response.raise_for_status()  # Raise an error if the request fails

    login_response = response.json()
    login_session_id = login_response.get("loginSessionId")
    user_id = login_response.get("userId")

    if not login_session_id or not user_id:
        raise ValueError("Failed to retrieve loginSessionId and userId from login response.")

    print("Authentication successful!")
    return login_session_id, user_id

# Step 3: Define the function to retrieve projects
def get_projects(login_session_id, user_id):
    """
    Retrieves a list of projects for the authenticated user and returns the first project's projectId.
    """
    projects_url = "https://consulting.us-east-1.reveal11.cloud/rest/api/v2/projects"

    headers = {
        "Authorization": f"Bearer {login_session_id}",
        "incontrolauthtoken": login_session_id,
        "User-Agent": "Reveal-API-Tester/1.0",
        "Accept": "application/json"
    }

    params = {"userId": user_id}
    response = requests.get(projects_url, headers=headers, params=params)
    response.raise_for_status()

    projects_response = response.json()
    projects = projects_response.get("results", [])

    if not projects:
        print("No projects found.")
        return None

    # Extract the projectId of the first project
    first_project = projects[0]
    project_id = first_project.get("projectId")
    project_name = first_project.get("projectName", "Unnamed Project")
    print(f"First project found: {project_name} (projectId: {project_id})")
    return project_id

# Step 4: Define the function to get users for a project
def get_project_users(login_session_id, project_id):
    """
    Retrieves a list of users for the specified projectId.
    """
    users_url = f"https://consulting.us-east-1.reveal11.cloud/rest/api/v2/{project_id}/users"

    headers = {
        "Authorization": f"Bearer {login_session_id}",
        "incontrolauthtoken": login_session_id,
        "User-Agent": "Reveal-API-Tester/1.0",
        "Accept": "application/json"
    }

    response = requests.get(users_url, headers=headers)
    response.raise_for_status()

    users_response = response.json()
    users = users_response.get("results", [])

    if users:
        print("Users in the project:")
        for user in users:
            print(f"- {user.get('userName', 'Unknown')} (ID: {user.get('userId')})")
    else:
        print("No users found for the project.")

# Main Execution
try:
    # Step 1: Authenticate to get session ID and user ID
    session_id, user_id = authenticate(username, password)

    # Step 2: Retrieve the first project ID
    project_id = get_projects(session_id, user_id)

    # Step 3: If project_id is found, retrieve and print users
    if project_id:
        get_project_users(session_id, project_id)

except requests.exceptions.RequestException as e:
    print_color(f"API request failed: {e}", "red")
except ValueError as e:
    print_color(f"Error: {e}", "red")


Authentication successful!
First project found: DanProject2 (projectId: 1795)
Users in the project:
- dhonors (ID: 372)
- dhonors2@revealdata.com (ID: 1715)
- msa81010002_reviewapi (ID: 262)
